In [1]:
############ Reprendre avec les codes de autoscout24

In [2]:
##Commencement : importer les packages

import requests 
import pandas as pd
import json   
from bs4 import BeautifulSoup 
from urllib.request import urlopen
import re


In [3]:
website = "https://www.bymycar.fr/v/voiture-occasion?itemsperpage=24&sortingorder=ascending&filter=marque-du-vehicule%3Amercedes"

In [4]:
response = requests.get(website)

In [5]:
soup = BeautifulSoup(response.content,"html.parser")

In [6]:
results = soup.find_all("a",{"class" : "product-item" })

In [7]:
len(results)

24

In [8]:
############## création boucle pour récupérer sur le véhicule :


# Marque 

# Modele

# Info vehicule 

# Prix

# Nombre de kilométrage 

# Année 

# puissance_chevaux


### Création d'une boucle pour récupérer les informations en ciblant les balises et les classes

In [9]:
modele = []
detail = []
prix = []
type_v = []
boite_v = []

for i in range(1,10):
    
    # website dans une variable
    website = "https://www.bymycar.fr/v/voiture-occasion?filter=marque-du-vehicule%3Amercedes&sortingorder=ascending&itemsperpage=24&display=grid&pagenumber="+str(i)

    # resquest au website
    response = requests.get(website)
    
    # soup object
    soup = BeautifulSoup(response.content,"html.parser")
    
    #result
    results = soup.find_all("a",{"class" : "product-item"})
    

    for result in results :

      try:

        modele.append(result.find("h2",{"class":"product__title"}).get_text())

      except:
        modele.append('n/a')

      try:

        detail.append(result.find("p",{"class" : "product__type" }).get_text())

      except:
        detail.append('n/a')  

      try:

        prix.append(result.find("div",{"class":"product__price_current"}).get_text())

      except:
        prix.append('n/a')

      try:

        type_v.append(result.find("ul",{"class" : "product__characteristics" }).get_text())

      except:
        type_v.append('n/a')      

      try:

        boite_v.append(result.find("ul",{"class" : "product__features_list" }).get_text())

      except:
        boite_v.append('n/a')    


In [10]:
car_dealer = pd.DataFrame({"Modele":modele , "Info vehicule":detail ,"Prix" :prix,
                           "type_vehicule":type_v,"puissance":boite_v})

### clean les colonnes du dataframe
-------------- Nettoyage de la base de données :


In [11]:
car_dealer["type_vehicule"] = car_dealer["type_vehicule"].str.lstrip("\n")

In [12]:
car_dealer["Nombre de kilometre"] = car_dealer["type_vehicule"].str.split("km", expand = True)[0]

In [13]:
def extract_after_word(string, word):
    if word in string:
        return string[string.index(word)+len(word):string.index(word)+len(word)+16]
    else:
        return None

In [14]:
car_dealer['Annee_sortie'] = car_dealer['type_vehicule'].apply(lambda x: extract_after_word(x,"km"))

In [15]:
def keep_digits(string):
    return ''.join(filter(str.isdigit, string))

In [16]:
car_dealer['Annee_sortie'] = car_dealer['Annee_sortie'].apply(keep_digits)

In [17]:
del car_dealer["type_vehicule"]

In [18]:
def extract_before_word(string, word):
    if word in string:
        return string[string.index(word)-4:string.index(word)]
    else:
        return None

In [19]:
car_dealer['puissance_chevaux'] = car_dealer['puissance'].apply(lambda x: extract_before_word(x, "cv"))

In [20]:
del car_dealer["puissance"]

In [21]:
car_dealer['Marque'] = "Mercedes"

In [22]:
car_dealer = car_dealer.reindex(columns=['Marque', 'Modele', 'Info vehicule','Prix', 'Nombre de kilometre', 'Annee_sortie',
                                         'puissance_chevaux'])


In [23]:
car_dealer['Prix'] = car_dealer['Prix'].apply(keep_digits)

In [26]:
car_dealer

,Marque,Modele,Info vehicule,Prix,Nombre de kilometre,Annee_sortie,puissance_chevaux
0,Mercedes,GLC 400 d 9G-Tronic 4Matic,DIESEL,66949,22477,2019,330
1,Mercedes,VITO FOURGON 119 CDI LONG BVA RWD,DIESEL,40739,73236,2019,190
2,Mercedes,Classe GLC 300 de 9G-Tronic 4Matic,Hybride,69900,10000,2022,194
3,Mercedes,Classe B 200 d 8G-DCT,DIESEL,33990,62568,2000,150
4,Mercedes,Classe A 200 7G-DCT,Essence,27968,50599,2018,163
...,...,...,...,...,...,...,...
211,Mercedes,GLC 300 de 9G-Tronic 4Matic,Hybride,72900,10000,2022,194
212,Mercedes,Classe A 220 d 8G-DCT,DIESEL,36176,32760,2020,190
213,Mercedes,Classe A 250 e 8G-DCT,Hybride,59200,0,2022,160
214,Mercedes,GLC 300 d 9G-Tronic 4Matic,DIESEL,61289,34000,2021,245


In [25]:
car_dealer.to_excel("Bymycar_excel.xlsx", index= False)